In [ ]:
gameId = "doapp"
#gameId = "sfiii3n"
#gameId = "tektagt"

In [ ]:
import sys, os
import time
timeDepSeed = int((time.time()-int(time.time()-0.5))*1000)

sys.path.append(os.path.join(os.path.abspath(''), '../'))   
sys.path.append(os.path.join(os.path.abspath(''), '../../../games'))   
sys.path.append(os.path.join(os.path.abspath(''), '../../../utils')) 

tensorBoardFolder = "./{}_ppo2_TB_CustCnnSmall_spL_d_comb/".format(gameId)
modelFolder = "./{}_ppo2_Model_CustCnnSmall_spL_d_comb/".format(gameId)

os.makedirs(modelFolder, exist_ok=True)

In [ ]:
from makeDiambraEnv import *

import tensorflow as tf

from customPolicies.utils import linear_schedule, AutoSave, UpdateRLPolicyWeights
from customPolicies.customCnnPolicy import *
from policies import noActionPolicy, randomPolicy, RLPolicy

from stable_baselines import PPO2

In [ ]:
model = PPO2.load(modelFolder + "6_5M")    

deterministicFlag = False                                                    
rl_policy = RLPolicy(model, deterministicFlag, [9, 8], name="PPO-6_5M", actionSpace="discrete")    

In [ ]:
# Diambra environment kwargs
diambraKwargs = {}
diambraKwargs["roms_path"]   = "../../../roms/mame/"
diambraKwargs["binary_path"] = "../../../customMAME/"
diambraKwargs["frame_ratio"] = 4
diambraKwargs["render"]      = False

diambraKwargs["player"] = "P1P2" # 2P game

# Game dependent kwawrgs
if gameId == "doapp":
    diambraKwargs["difficulty"]  = 3 
    diambraKwargs["characters"] =["Random", "Random"]
    diambraKwargs["charOutfits"] =[2, 2]
elif gameId == "sfiii3n":
    diambraKwargs["difficulty"]  = 6
    diambraKwargs["characters"] =["Random", "Random"]
    diambraKwargs["charOutfits"] =[2, 2]
elif gameId == "tektagt":
    diambraKwargs["difficulty"]  = 6
    diambraKwargs["characters"] =[["Random", "Random"], ["Random", "Random"]]
    diambraKwargs["charOutfits"] =[2, 2]
else:
    raise Exception("Game not implemented: {}".format(gameId))

# DIAMBRA gym kwargs
diambraGymKwargs = {}
diambraGymKwargs["P2brain"] = rl_policy
diambraGymKwargs["continue_game"] = 0.0 # If < 0.0 means number of continues
diambraGymKwargs["show_final"] = False
diambraGymKwargs["gamePads"] = [None, None]                                     
diambraGymKwargs["actionSpace"] = ["discrete", "discrete"]
diambraGymKwargs["attackButCombinations"] = [True, True]

# Gym Wrappers kwargs
wrapperKwargs = {}
wrapperKwargs["hwc_obs_resize"] = [128, 128, 1]
wrapperKwargs["normalize_rewards"] = True
wrapperKwargs["clip_rewards"] = False
wrapperKwargs["frame_stack"] = 4
wrapperKwargs["dilation"] = 1
wrapperKwargs["scale"] = True
wrapperKwargs["scale_mod"] = 0

# Additional observations
keyToAdd = []
keyToAdd.append("actionsBuf") # env.actBufLen*(env.n_actions[0]+env.n_actions[1])

if gameId != "tektagt":                                                         
    keyToAdd.append("ownHealth")   # 1                                            
    keyToAdd.append("oppHealth")   # 1                                                
else:                                                                           
    keyToAdd.append("ownHealth_1") # 1                                             
    keyToAdd.append("ownHealth_2") # 1                                             
    keyToAdd.append("oppHealth_1") # 1                                              
    keyToAdd.append("oppHealth_2") # 1  
    
keyToAdd.append("ownPosition")     # 1
keyToAdd.append("oppPosition")     # 1
keyToAdd.append("character")       # len(env.charNames)

numEnv=8

envId = gameId + "_Train"
env = make_diambra_env(diambraMame, env_prefix=envId, num_env=numEnv, seed=timeDepSeed, 
                       diambra_kwargs=diambraKwargs, diambra_gym_kwargs=diambraGymKwargs,
                       wrapper_kwargs=wrapperKwargs, key_to_add=keyToAdd, use_subprocess=False)

In [ ]:
print("Obs_space = ", env.observation_space)
print("Obs_space type = ", env.observation_space.dtype)
print("Obs_space high = ", env.observation_space.high)
print("Obs_space low = ", env.observation_space.low)

In [ ]:
print("Act_space = ", env.action_space)
print("Act_space type = ", env.action_space.dtype)
if diambraGymKwargs["actionSpace"][0] == "multiDiscrete":
    print("Act_space n = ", env.action_space.nvec)
else:
    print("Act_space n = ", env.action_space.n)

In [ ]:
# Policy param
n_actions = env.get_attr("n_actions")[0][0]
actBufLen = env.get_attr("actBufLen")[0]
nChar = env.get_attr("numberOfCharacters")[0]

policyKwargs={}
policyKwargs["n_add_info"] = actBufLen*(n_actions[0]+n_actions[1]) + len(keyToAdd)-2 + nChar
policyKwargs["layers"] = [64, 64]

policyKwargs["cnn_extractor"] = local_nature_cnn_small

print("n_actions =", n_actions)
print("nChar =", nChar)
print("n_add_info =", policyKwargs["n_add_info"])

In [ ]:
# PPO param
setGamma = 0.94
setLearningRate = linear_schedule(2.5e-4, 2.5e-6)
#setLearningRate = linear_schedule(5.0e-5, 2.5e-6)
setClipRange = linear_schedule(0.15, 0.025)
#setClipRange = linear_schedule(0.05, 0.025)
setClipRangeVf = setClipRange

# Initialize the model
#model = PPO2(CustCnnPolicy, env, verbose=1, 
#             gamma = setGamma, nminibatches=4, noptepochs=4, n_steps=128,
#             learning_rate=setLearningRate, cliprange=setClipRange, cliprange_vf=setClipRangeVf, 
#             tensorboard_log=tensorBoardFolder, policy_kwargs=policyKwargs)

#OR

# Load the trained agent
model = PPO2.load(modelFolder+"6_5M", env=env, tensorboard_log=tensorBoardFolder, 
                  policy_kwargs=policyKwargs, gamma = setGamma, learning_rate=setLearningRate, 
                  cliprange=setClipRange, cliprange_vf=setClipRangeVf)

In [ ]:
print("Model discount factor = ", model.gamma)

In [ ]:
# Create the callback: autosave every USER DEF steps
autoSaveCallback = AutoSave(check_freq=1000000, numEnv=numEnv, save_path=modelFolder+"6_5M_")

prevAgentsSamplingDict = {"probability": 0.3,
                          "list":[modelFolder + "6_5M", modelFolder + "13M"]}
upRLPolWCallback = UpdateRLPolicyWeights(check_freq=10000, numEnv=numEnv, save_path=modelFolder,
                                         prevAgentsSampling=prevAgentsSamplingDict)

# Train the agent
time_steps = 30000000
model.learn(total_timesteps=time_steps, callback=[autoSaveCallback, upRLPolWCallback])

In [ ]:
# Save the agent
model.save(modelFolder+"13M")